In [1]:
!pip install newsapi-python google-generativeai

In [4]:
import pandas as pd
from newsapi import NewsApiClient

# --- Sua Chave da News API ---
NEWS_API_KEY = 'a832cf7fe7cf4ab28fae37a4c9a50415'

newsapi = NewsApiClient(api_key=NEWS_API_KEY)

# --- Domínios para Filtragem de Fontes ---
dominios = "infomoney.com.br, valor.globo.com, braziljournal.com, mckinsey.com.br"

# --- Queries por Categoria de Risco ---
queries_por_categoria = {
    "Ambientais_Climaticos": "(('mudanças climáticas' OR 'crise climática' OR 'aquecimento global') AND ('risco' OR 'impacto')) OR 'eventos climáticos extremos' OR 'seca severa' OR 'inundação' OR 'ondas de calor' OR 'incêndios florestais' OR 'perda de biodiversidade' OR 'escassez de água' OR 'poluição' OR 'transição energética' OR 'descarbonização' OR ESG",
    "Geopoliticos_Sociais": "('conflito armado' OR 'tensão geopolítica' OR 'instabilidade política' OR 'guerra comercial' OR 'sanções') OR ('polarização social' OR 'fragmentação social' OR 'desigualdade social') OR ('desinformação' OR 'fake news') OR 'migração em massa' OR 'saúde pública' OR 'pandemia' OR 'segurança alimentar' OR 'protestos'",
    "Tecnologicos": "('inteligência artificial' OR IA OR 'riscos da IA' OR 'IA generativa') AND ('ética' OR 'segurança' OR 'regulamentação' OR 'impacto social' OR 'viés') OR 'cibersegurança' OR 'ataque cibernético' OR 'ransomware' OR 'vazamento de dados' OR 'disrupção digital' OR 'novas tecnologias' OR 'biotecnologia' OR 'computação quântica' OR 'deepfake' OR 'privacidade de dados'",
    "Economicos": "inflação OR recessão OR 'crise econômica' OR 'instabilidade financeira' OR 'dívida pública' OR 'bolha de ativos' OR 'cadeia de suprimentos' OR 'interrupção logística' OR 'escassez de matéria-prima' OR 'desemprego estrutural' OR 'crise de custo de vida' OR 'choque de oferta' OR 'protecionismo econômico'",
    "Governanca_Resiliencia": "'falha de governança' OR 'governança corporativa' OR 'resiliência operacional' OR 'crise de reputação' OR 'conformidade regulatória' OR 'risco ESG' OR 'gestão de crise' OR 'fragilidade institucional' OR 'ameaça iminente' OR 'desafios futuros' OR 'tendências de risco' OR 'crise potencial'"
}

# Lista para armazenar todos os artigos coletados
all_articles = []
processed_urls = set()

# --- Loop para Coletar Notícias por Categoria ---
print("--- Iniciando Coleta de Notícias da News API ---")
for category, query_str in queries_por_categoria.items():
    print(f"Buscando notícias para a categoria: {category}")

    try:
        response = newsapi.get_everything(
            q=query_str,
            domains=dominios,
            language='pt',
            sort_by='relevancy',
            page_size=100,
            from_param='2025-05-11'
        )

        if response['status'] == 'ok':
            for article in response['articles']:
                # Verifica se o artigo já foi adicionado para evitar duplicatas
                if article['url'] not in processed_urls:
                    # Adiciona uma nova chave 'risk_category_newsapi' ao artigo
                    # para indicar qual query da News API o encontrou.
                    article['risk_category_newsapi'] = category
                    all_articles.append(article)
                    processed_urls.add(article['url'])
        else:
            print(f"Erro na requisição para {category}: {response['code']} - {response['message']}")

    except Exception as e:
        print(f"Ocorreu um erro ao buscar notícias para {category}: {e}")

# --- Criação do DataFrame Pandas ---
df_noticias = pd.DataFrame(all_articles)

print(f"\n--- Coleta de Notícias Concluída ---")
print(f"Total de artigos únicos coletados pela News API: {len(df_noticias)}")

if not df_noticias.empty:
    print("\nVisualização do DataFrame das notícias coletadas:")
    print(df_noticias[['title', 'source', 'description', 'risk_category_newsapi']].head())
else:
    print("Nenhum artigo foi coletado. O DataFrame está vazio.")

--- Iniciando Coleta de Notícias da News API ---
Buscando notícias para a categoria: Ambientais_Climaticos
Buscando notícias para a categoria: Geopoliticos_Sociais
Buscando notícias para a categoria: Tecnologicos
Buscando notícias para a categoria: Economicos
Buscando notícias para a categoria: Governanca_Resiliencia

--- Coleta de Notícias Concluída ---
Total de artigos únicos coletados pela News API: 136

Visualização do DataFrame das notícias coletadas:
                                               title  \
0  “Negação econômica” é risco na luta contra mud...   
1  Tanque oceânico, mecânica computacional e uso ...   
2  Preço do café sobe 80% e consumo no Brasil reg...   
3  Congresso analisa vetos presidenciais à Lei da...   
4  Quase todos os biomas registraram redução do d...   

                                      source  \
0  {'id': 'info-money', 'name': 'InfoMoney'}   
1  {'id': 'info-money', 'name': 'InfoMoney'}   
2  {'id': 'info-money', 'name': 'InfoMoney'}   
3  {'id': 

In [10]:
df_noticias[['title', 'source', 'url', 'description', 'content', 'risk_category_newsapi']]

,title,source,url,description,content,risk_category_newsapi
0,“Negação econômica” é risco na luta contra mud...,"{'id': 'info-money', 'name': 'InfoMoney'}",https://www.infomoney.com.br/mundo/negacao-eco...,"Em entrevista ao The Guardian, diplomata brasi...","À medida que o mundo se prepara para a COP30, ...",Ambientais_Climaticos
1,"Tanque oceânico, mecânica computacional e uso ...","{'id': 'info-money', 'name': 'InfoMoney'}",https://www.infomoney.com.br/brasil/tanque-oce...,Uma das iniciativas simula principais caracter...,Um levantamento feito pelo Center for World Un...,Ambientais_Climaticos
2,Preço do café sobe 80% e consumo no Brasil reg...,"{'id': 'info-money', 'name': 'InfoMoney'}",https://www.infomoney.com.br/consumo/preco-do-...,"Alta é a maior em 30 anos; impacto da seca, cu...","O consumo de café no Brasil caiu 15,9% em abri...",Ambientais_Climaticos
3,Congresso analisa vetos presidenciais à Lei da...,"{'id': 'info-money', 'name': 'InfoMoney'}",https://www.infomoney.com.br/colunistas/convid...,A derrubada desses vetos pode provocar um impa...,"O Brasil está prestes a tomar uma decisão que,...",Ambientais_Climaticos
4,Quase todos os biomas registraram redução do d...,"{'id': 'info-money', 'name': 'InfoMoney'}",https://www.infomoney.com.br/brasil/quase-todo...,"Na comparação com 2023, a redução foi de 32,4%...",Cinco dos seis biomas brasileiros tiveram redu...,Ambientais_Climaticos
...,...,...,...,...,...,...
131,Impacto do IOF nas empresas é de R$ 20 mil a c...,"{'id': 'info-money', 'name': 'InfoMoney'}",https://www.infomoney.com.br/economia/impacto-...,"Remessas ao exterior ficaram R$ 17,7 mil mais ...",O aumento do Imposto sobre Operações Financeir...,Economicos
132,Encerramento do QuintoCred expõe fragilidade n...,"{'id': 'info-money', 'name': 'InfoMoney'}",https://www.infomoney.com.br/minhas-financas/e...,"Para especialistas, medida reacende debate sob...",O anúncio do encerramento das operações do Qui...,Governanca_Resiliencia
133,PRIO (PRIO3) registra melhora da produção em m...,"{'id': 'info-money', 'name': 'InfoMoney'}",https://www.infomoney.com.br/mercados/prio-pri...,PRIO (PRIO3) divulgou nesta quarta-feira (4) u...,A PRIO (PRIO3) divulgou nesta quarta-feira (4)...,Governanca_Resiliencia
134,"Petrobras, Vale, Lojas Renner, Allied e mais a...","{'id': 'info-money', 'name': 'InfoMoney'}",https://www.infomoney.com.br/mercados/petrobra...,Confira os principais destaques do noticiário ...,O radar corporativo desta terça-feira (27) tra...,Governanca_Resiliencia
